# Trabajo Práctico 5

In [12]:
import cv2 as cv
import random
import numpy as np
import time

In [15]:
def getMedianImage(capture, n: int):
    # Obtengo los IDs de n frames aleatorios
    frameIds = capture.get(cv.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=n)
    
    # Guardo los frames en una lista
    frames = []
    for id in frameIds:
        capture.set(cv.CAP_PROP_POS_FRAMES, id)
        ret, frame = capture.read()
        frames.append(frame)

    # Calculo la mediana
    medianFrame = np.median(frames, axis=0).astype(dtype=np.uint8)    
    return medianFrame

def NaiveBackgroundSubstractor(capture, n: int, segundos: int):


    grayMedianFrame = cv.cvtColor(getMedianImage(capture, n), cv.COLOR_BGR2GRAY)

    # Volvemos al primer frame del video
    capture.set(cv.CAP_PROP_POS_FRAMES, 0)

    #Inicializamos el temporizador
    startTime = time.time()
    
    ret = True
    while(ret):

        if (time.time() - startTime) > segundos:
            grayMedianFrame = cv.cvtColor(getMedianImage(capture, n), cv.COLOR_BGR2GRAY)
            startTime =  time.time()
            print("Se ha actualizado el fondo")
        # Leemos cada frame
        ret, frame = capture.read()
        frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        # Restamos el frame con la mediana
        dframe = cv.absdiff(frame, grayMedianFrame)
        # Aplicamos un threshold para binarizar
        th, dframe = cv.threshold(dframe, 30, 255, cv.THRESH_BINARY)
        # Display image
        cv.imshow('frame', dframe)

        keyboard = cv.waitKey(30)
        if keyboard == 'q' or keyboard == 27:
              break
        
    # Release video object
    capture.release()
    
    # Destroy all windows
    cv.destroyAllWindows()


In [19]:
filename = 'vtest.avi'
capture = cv.VideoCapture(filename)

NaiveBackgroundSubstractor(capture, 15, 3)

Se ha actualizado el fondo
Se ha actualizado el fondo
Se ha actualizado el fondo
Se ha actualizado el fondo


Comparando con la substraccion de fondo basado en mezcla de gausianas, se puede notar que usar la mediana como estimador puede generarnos un video menos fluido (con pausas para actualizar el fondo), aunque la sustracción del fondo parece ser mejor.